In [33]:
!pip install onnx onnxruntime

In [ ]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
shape_output_name = "Y"

# Create the ONNX model with Shape operator
def create_shape_model(input_rank, start, end, dtype):

    #Create input tensor
    input1 = onnx.helper.make_tensor_value_info(input_name, dtype, input_rank)

    # Create output tensor (final result after shape operation)
    shape_output = onnx.helper.make_tensor_value_info(shape_output_name, onnx.TensorProto.INT64, [None])

    # Define shape node
    shape_node = onnx.helper.make_node(
        "Shape",
        inputs=[input_name],
        outputs=[shape_output_name],
        start=start,
        end=end
    )

    # Create the ONNX graph
    graph_def = onnx.helper.make_graph(
        [shape_node],
        "Shape",
        [input1],
        [shape_output],
    )

    # Create the ONNX model
    model = onnx.helper.make_model(graph_def, opset_imports=[onnx.helper.make_opsetid("", 22)]) # Explicitly set opset to 22
    model.ir_version = 10 
    onnx.checker.check_model(model)

    # Save the model
    onnx.save(model, "shape.onnx")

    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("shape.onnx")
    return session

def do_shapes(x, session):
    # Run inference
    output = session.run(None, {input_name: x})

    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', ''))
    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print("Shape of input tensor:", x.shape)
    print(f"X={x_f}")
    print("Shape of output tensor:", output[0].shape)
    print(f"Result = {y_f}")


np.set_printoptions(precision=None, floatmode='fixed')

## Nominal Cases

In [2]:
# Case N1: 3-rank tensor (int32), start = 0, end = 3
onnx_type = onnx.TensorProto.INT32
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
input_shape = x.shape  
start = 0
end = 3
session = create_shape_model([None,None,None], start, end, onnx_type)
do_shapes(x, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (3,)
Result = [2,3,4]


In [3]:
# Case N2: 3-rank tensor (int32), start = 1, end = 2
onnx_type = onnx.TensorProto.INT32
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
input_shape = x.shape  
start = 1
end = 2
session = create_shape_model([None,None,None], start, end, onnx_type)
do_shapes(x, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (1,)
Result = [3]


In [4]:
# Case N3: 3-rank tensor (int32), start = 2, end = 2
onnx_type = onnx.TensorProto.INT32
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
input_shape = x.shape  
start = 2
end = 2
session = create_shape_model([None,None,None], start, end, onnx_type)
do_shapes(x, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (0,)
Result = []


In [5]:
# Case N4: 3-rank tensor (int32), start = -500, end = 2
onnx_type = onnx.TensorProto.INT32
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
input_shape = x.shape  
start = -500
end = 2
session = create_shape_model([None,None,None], start, end, onnx_type)
do_shapes(x, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (2,)
Result = [2,3]


In [6]:
# Case N5: 3-rank tensor (int32), start = 0, end = 1000
onnx_type = onnx.TensorProto.INT32
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
input_shape = x.shape  
start = 0
end = 1000
session = create_shape_model([None,None,None], start, end, onnx_type)
do_shapes(x, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (3,)
Result = [2,3,4]


In [7]:
# Case N6: 3-rank tensor (int32), start = -500, end = -400
onnx_type = onnx.TensorProto.INT32
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
input_shape = x.shape  
start = -500
end = -400
session = create_shape_model([None,None,None], start, end, onnx_type)
do_shapes(x, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (0,)
Result = []


In [8]:
# Case N7: 3-rank tensor (int32), start = 400, end = 500
onnx_type = onnx.TensorProto.INT32
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
input_shape = x.shape  
start = 400
end = 500
session = create_shape_model([None,None,None], start, end, onnx_type)
do_shapes(x, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (0,)
Result = []
